In [37]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("postoffice.csv")

columns_to_remove = ['alt_name', 'discontinued', 'county1', 'county2', 'county3',  'id', 'gnis_name', 'gnis_match', 'gnis_county', 'gnis_state', 'gnis_feature_id', 'gnis_feature_class', 'gnis_orig_name', 'gnis_orig_county', 'gnis_latitude', 'gnis_longitude', 'gnis_elev_in_m', 'gnis_dist']

# Remove the specified columns
df_cleaned = df.drop(columns=columns_to_remove)
df_cleaned = df_cleaned.dropna(subset=['latitude', 'longitude', 'established'])
# Keep only the current running offices
df_cleaned = df_cleaned[df_cleaned['duration'].isna()]
# Calculate 'duration' based on 'established' column if it's missing
df_cleaned['duration'] = df_cleaned.apply(lambda row: (2021 - row['established']) if pd.isnull(row['duration']) else row['duration'], axis=1)

# Check for non-convertible values in 'established' column
non_convertible_established = df_cleaned[df_cleaned['established'].isnull()]['established']
print("Non-convertible values in 'established' column:")
print(len(df_cleaned))

df_cleaned_sorted = df_cleaned.sort_values(by='established', ascending=False)

# Keep only the top 1000 recent post offices
df_top_1000_recent = df_cleaned_sorted.head(1000)

# Check the first few rows of the resulting DataFrame
print(df_top_1000_recent.head())


Non-convertible values in 'established' column:
28676
                      name            orig_name state     orig_county  \
80712             ELKRIDGE         ELKRIDGE (2)    MD          Howard   
37044          MIAMI BEACH          MIAMI BEACH    FL            Dade   
83431      FORT WASHINGTON  FORT WASHINGTON (2)    MD  Prince Georges   
3309   BATESBURG-LEESVILLE  BATESBURG-LEESVILLE    SC       Lexington   
43810           PARK HILLS           PARK HILLS    MO    St. Francois   

       established  continuous  stamp_index  coordinates  duration   latitude  \
80712       2000.0        True          0.0         True      21.0  39.190580   
37044       1999.0        True          0.0         True      22.0  25.784469   
83431       1996.0        True          0.0         True      25.0  38.730840   
3309        1995.0        True          0.0         True      26.0  33.916532   
43810       1995.0        True          0.0         True      26.0  37.850758   

       longitude  
8

In [38]:

# Convert 'duration' and 'established' columns to integers
df_top_1000_recent['duration'] = df_top_1000_recent['duration'].astype(int)
df_top_1000_recent['established'] = df_top_1000_recent['established'].astype(int)


# Write the cleaned data to a new CSV file
df_top_1000_recent.to_csv("postoffice_cleaned_top1000.csv", index=False)

print("Rows with NA values removed. Cleaned data saved to postoffice_cleaned.csv")


Rows with NA values removed. Cleaned data saved to postoffice_cleaned.csv


/var/folders/mt/rp9xph9j5nvb8zt52znkhhr00000gn/T/ipykernel_46903/515352458.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_1000_recent['duration'] = df_top_1000_recent['duration'].astype(int)
/var/folders/mt/rp9xph9j5nvb8zt52znkhhr00000gn/T/ipykernel_46903/515352458.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_1000_recent['established'] = df_top_1000_recent['established'].astype(int)


In [39]:
import pandas as pd

# Read the cleaned CSV file
df_cleaned = pd.read_csv("postoffice_cleaned_top1000.csv")

# Dictionary mapping short names to long names of states
state_names_long = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Invert the dictionary to get short names as keys and long names as values
state_names_short = {v: k for k, v in state_names_long.items()}

# Map short state names to long state names
df_cleaned['state_longform'] = df_cleaned['state'].map(state_names_long)

# Aggregate number of post offices per state
state_post_offices_long = df_cleaned.groupby('state_longform').size().reset_index(name='num_post_offices')

# Map long state names back to short state names
state_post_offices_long['state'] = state_post_offices_long['state_longform'].map(state_names_short)

# Concatenate short and long state names with aggregated number of post offices
state_post_offices = pd.concat([state_post_offices_long[['state', 'state_longform', 'num_post_offices']], state_post_offices_long[['state', 'state_longform', 'num_post_offices']]])

# Calculate the centroid of each state and save it to the CSV
centroid_latitudes = []
centroid_longitudes = []

for state_longform in state_post_offices['state_longform']:
    state_data = df_cleaned[df_cleaned['state_longform'] == state_longform]
    centroid_latitude = state_data['latitude'].mean()
    centroid_longitude = state_data['longitude'].mean()
    centroid_latitudes.append(centroid_latitude)
    centroid_longitudes.append(centroid_longitude)

state_post_offices['centroid_latitude'] = centroid_latitudes
state_post_offices['centroid_longitude'] = centroid_longitudes

# Write the aggregated data to a new CSV file
state_post_offices.to_csv("statewise_post_offices_top1000.csv", index=False)

print("State-wise post office data saved to statewise_post_offices.csv")


State-wise post office data saved to statewise_post_offices.csv
